# Projeto 2 - E-Redes - Transição Energética

Análise dos dados em bruto para detetar paadrões e formas de enriquecer os dados

# Análise exploratória

Indicadores:
- Autoconsumo
    - A1: Total de unidades de produção para autoconsumo
    - A2: Novas unidades de produção para Autoconsumo
    - A3: Ligações à rede
    - A4: Total de unidades de produção para autoconsumo - Auxiliar

- Energias Renováveis
    - E1: Energia Injetada na Rede de Distribuição
    - E2: Energia Produzida Total Nacional

- Iluminação Pública
    - **I1: Caracterização de luminárias de Iluminação Pública**

            - Total de 9.434 registos;
            - Tem 1015 valores nulos na coluna "potência instalada";
            - Organizado por Distrito, concelho e/ou freguesia;
            - Tipo de lâmpada: 
                    - Sódio;
                    - Outros/Não definido;
                    - LED;
                    - Mercúrio;
                    - Iodeto Metálico;
                    - Indução;
                    - Fluorescente.
            - Tempo: Março de 2025 (?) perguntar se estes dados so começaram a ser disponibilizados em março de 2025

    - **I2: Caracterização de Pontos de Consumo (CPEs), com contratos ativos**

            Características dos CPEs por:
                - Distrito, concelho ou freguesia;
                - Tipo de instalação: Outros, Não doméstico, Doméstico e Iluminação Pública;
                - Entre janeiro 2022 e fevereiro 2025
    
- Contadores inteligentes
    - C1: Número de contratos de energia ativos por tipo de contador
    - C2: Número de locais de consumo de baixa tensão com Leituras Remotas

- Mobilidade Eléctrica
    - M1: Novas ligações à rede associados à mobilidade elétrica
    - M2: Número de locais de consumo de baixa tensão com recolha de Diagramas de Carga
    - M3: Pontos de ligação para postos de Carregamento de Veículos Elétricos
    
    
    


# Setup

Configurações para correr o código

In [2]:
import pandas as pd 
import numpy as np 
import os 

In [7]:
# Garantir diretorias

print(os.getcwd())  # Prints the current working directory
os.chdir('C:\Files\GIThub_projects\ITTE')  # Changes the working directory

c:\Files\GIThub_projects\ITTE\notebooks


<>:4: SyntaxWarning: invalid escape sequence '\F'
<>:4: SyntaxWarning: invalid escape sequence '\F'
C:\Users\magda\AppData\Local\Temp\ipykernel_16536\921853709.py:4: SyntaxWarning: invalid escape sequence '\F'
  os.chdir('C:\Files\GIThub_projects\ITTE')  # Changes the working directory


# Autoconsumo

In [26]:
dfA1 = pd.read_csv(r'data/Autoconsumo/8-unidades-de-producao-para-autoconsumo.csv', delimiter=';')

dfA2 = pd.read_csv(r'data\Autoconsumo\16-pedidos-concluidos-plrs.csv', delimiter = ';')

dfA3 = pd.read_csv(r'data\Autoconsumo\26-centrais.csv', delimiter=';')

dfA4 = pd.read_csv(r'data\Autoconsumo\total-de-unidades-de-producao-para-autoconsumo-auxiliar.csv', delimiter=';')

# Energias Renováveis

In [27]:
dfE1 = pd.read_csv(r'data/Energias_renovaveis/energia-injetada-na-rede-de-distribuicao.csv', delimiter=';')

dfE2 = pd.read_csv(r'data/Energias_renovaveis/energia-produzida-total-nacional.csv', delimiter=';')

# Iluminação Pública

In [31]:
dfI1 = pd.read_csv(r'data/Iluminacao_publica/cadastro_iluminacao_publica.csv', delimiter=';')

dfI2 = pd.read_csv(r'data/Iluminacao_publica/20-caracterizacao-pes-contrato-ativo.csv', delimiter=';')

## I1 - Caracterização de luminárias de Iluminação Pública

In [63]:
dfI1.shape

(9434, 11)

In [64]:
dfI1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9434 entries, 0 to 9433
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ano                           9434 non-null   int64  
 1   mes                           9434 non-null   int64  
 2   Distrito                      9434 non-null   object 
 3   Concelho                      9434 non-null   object 
 4   Freguesia                     9434 non-null   object 
 5   tipo_de_lampada               9434 non-null   object 
 6   luminarias                    9434 non-null   float64
 7   potencia instalada total (W)  8419 non-null   float64
 8   CodDistrito                   9434 non-null   int64  
 9   CodDistritoConcelho           9434 non-null   int64  
 10  coddistritoconcelhofreguesia  9434 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 810.9+ KB


In [66]:
dfI1.isnull().sum()

ano                                0
mes                                0
Distrito                           0
Concelho                           0
Freguesia                          0
tipo_de_lampada                    0
luminarias                         0
potencia instalada total (W)    1015
CodDistrito                        0
CodDistritoConcelho                0
coddistritoconcelhofreguesia       0
dtype: int64

In [33]:
#Dataframe: registos de iluminação pública
dfI1.columns

Index(['ano', 'mes', 'Distrito', 'Concelho', 'Freguesia', 'tipo_de_lampada',
       'luminarias', 'potencia instalada total (W)', 'CodDistrito',
       'CodDistritoConcelho', 'coddistritoconcelhofreguesia'],
      dtype='object')

In [37]:
# Tipos de lampada
dfI1['tipo_de_lampada'].unique()

array(['Sódio', 'Outros/Não definido', 'LED', 'Mercúrio',
       'Iodeto Metálico', 'Indução', 'Fluorescente'], dtype=object)

In [76]:
dfI1['ano'].unique()

array([2025])

In [ ]:
#Concatenar ano e mes
dfI1['Data']=(dfI1['ano'].astype(str) 
                + '-' 
                + dfI1['mes'].astype(str))

dfI1['Data']

0       2025-3
1       2025-3
2       2025-3
3       2025-3
4       2025-3
         ...  
9429    2025-3
9430    2025-3
9431    2025-3
9432    2025-3
9433    2025-3
Name: Data, Length: 9434, dtype: object

In [75]:
dfI1['Data'].unique()

array(['2025-3'], dtype=object)

## I2 - Caracterização de CPEs com contratos ativos

In [ ]:
dfI2.shape

(598165, 12)


In [ ]:
dfI2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598165 entries, 0 to 598164
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Ano                           598165 non-null  int64 
 1   mes                           598165 non-null  int64 
 2   Data                          598165 non-null  object
 3   Distrito                      598165 non-null  object
 4   Concelho                      598165 non-null  object
 5   Freguesia                     598165 non-null  object
 6   tipo_de_instalacao            598165 non-null  object
 7   nivel_de_tensao               598165 non-null  object
 8   cpes                          598165 non-null  int64 
 9   CodDistrito                   598165 non-null  int64 
 10  CodDistritoConcelho           598165 non-null  int64 
 11  coddistritoconcelhofreguesia  598165 non-null  object
dtypes: int64(5), object(7)
memory usage: 54.8+ MB
None


In [67]:
dfI2.isnull().sum()

Ano                             0
mes                             0
Data                            0
Distrito                        0
Concelho                        0
Freguesia                       0
tipo_de_instalacao              0
nivel_de_tensao                 0
cpes                            0
CodDistrito                     0
CodDistritoConcelho             0
coddistritoconcelhofreguesia    0
dtype: int64

In [38]:
dfI2['tipo_de_instalacao'].unique()

array(['Outros', 'Não Doméstico', 'Doméstico', 'Iluminação Pública'],
      dtype=object)

In [61]:
dfI2['Data'].unique()


array(['2022-08', '2024-12', '2024-02', '2022-05', '2022-01', '2024-11',
       '2024-05', '2024-08', '2024-06', '2024-10', '2022-10', '2022-06',
       '2023-06', '2023-05', '2023-02', '2023-03', '2024-07', '2022-03',
       '2023-07', '2024-01', '2023-01', '2025-01', '2022-12', '2024-09',
       '2023-04', '2022-04', '2025-02', '2023-11', '2022-09', '2024-04',
       '2023-08', '2022-11', '2023-12', '2023-09', '2022-07', '2022-02',
       '2024-03', '2023-10'], dtype=object)

In [62]:
dfI2 = dfI2.sort_values(by='Data')

dfI2.head(5)

,Ano,mes,Data,Distrito,Concelho,Freguesia,tipo_de_instalacao,nivel_de_tensao,cpes,CodDistrito,CodDistritoConcelho,coddistritoconcelhofreguesia
227902,2022,1,2022-01,Santarém,Torres Novas,"Torres Novas (São Pedro), Lapas e Ribeira Branca",Não Doméstico,Baixa Tensão Normal,831,14,1419,141921
305436,2022,1,2022-01,Santarém,Alcanena,Serra de Santo António,Doméstico,Baixa Tensão Normal,394,14,1402,140209
305437,2022,1,2022-01,Viseu,Moimenta da Beira,"Pêra Velha, Aldeia de Nacomba e Ariz",Iluminação Pública,Baixa Tensão Normal,5,18,1807,180722
305438,2022,1,2022-01,Viseu,Armamar,Aldeias,Iluminação Pública,Baixa Tensão Normal,3,18,1801,180101
305439,2022,1,2022-01,Aveiro,Arouca,Fermedo,Não Doméstico,Baixa Tensão Normal,115,1,104,010411


# Contadores inteligentes

In [17]:
dfC1=pd.read_csv(r'data\Contadores_inteligentes\21-contadores-de-energia.csv', delimiter=';')

dfC2=pd.read_csv(r'data/Contadores_inteligentes/23-leituras-recolhidas-remotamente.csv', delimiter=';')

In [20]:
# Dataframe C1: Contadores de energia
dfC1.columns

Index(['Ano', 'Mês', 'Data', 'Distrito', 'Concelho', 'Freguesia',
       'Inclui contador inteligente', 'Número de CPE's', 'Código Distrito',
       'Código Concelho', 'Código Freguesia', 'Contrato Ativo'],
      dtype='object')

In [21]:
# Dataframe C2: Leituras recolhidas remotamente
dfC2.columns

Index(['Ano', 'Mes', 'data', 'Distrito', 'Concelho', 'Freguesia',
       'CPEs com Leituras (#)', 'CodDistrito', 'CodDistritoConcelho',
       'CodDistritoConcelhoFreguesia'],
      dtype='object')

# Mobilidade Eléctrica

In [22]:
dfM1 = pd.read_csv(r'data/Mobilidade_eletrica/9-plr-mobilidade-eletrica.csv',delimiter=';')

dfM2 = pd.read_csv(r'data\Mobilidade_eletrica\22-diagrama-de-carga-por-instalacao.csv',delimiter=';')

dfM3 = pd.read_csv(r'data/Mobilidade_eletrica/postos_carregamento_ves.csv',delimiter=';')

In [23]:
#Dataframe M1: Pedidos de ligação

dfM1.columns

Index(['Ano', 'Semestre', 'Concelho',
       'Pedidos de Ligação à Rede Executados (#)', 'CodConcelho'],
      dtype='object')

In [24]:
#Dataframe M2: Diagrama de carga por instalação

dfM2.columns

Index(['Ano', 'Mês', 'data', 'Distrito', 'Concelho', 'Freguesia',
       'Inclui Mobilidade Elétrica', 'CPEs com DCs Recolhidos (#)',
       'CodDistrito', 'CodDistritoConcelho', 'CodDistritoConcelhoFreguesia'],
      dtype='object')

In [25]:
#Datafram M3: Postos de carregamento de VE

dfM3.columns

Index(['Trimestre', 'Distrito', 'Concelho', 'Freguesia',
       'Potência Máxima Admissível (kW)',
       'Pontos de ligação para instalações de PCVE', 'CodDistrito',
       'CodDistritoConcelho', 'CodDistritoConcelhoFreguesia'],
      dtype='object')